In [ ]:
import tifffile as tiff
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from keras.models import Model
from keras.layers import Input, Dense, Embedding, concatenate
from keras.layers import CuDNNGRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D
from keras.layers import Add, BatchNormalization, Activation, CuDNNLSTM, Dropout
from keras.layers import *
from keras.models import *
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
import matplotlib.pyplot as plt

import numpy as np
#import earthpy.plot as ep
import seaborn as sns
#import earthpy.spatial as es

import plotly.graph_objects as go
import plotly.express as px

from scipy.io import loadmat

import pandas as pd

from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score,
                             confusion_matrix, classification_report)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Conv1D,Conv2D, MaxPooling1D, Dropout, Flatten,BatchNormalization,MaxPooling2D
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

from tqdm import tqdm
from numpy.random import seed
from time import time
from keras.layers import concatenate
from keras.layers import Add
from tensorflow.keras.layers import Dropout, Input

# Transformer

In [ ]:
class PatchEncoder(tf.keras.layers.Layer):
    def __init__(self, num_patches,projection_dim = 256, **kwargs):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches  ## maxlen
        self.projection = layers.Dense(projection_dim) # head_size
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

    def get_config(self):
        config = super(PatchEncoder, self).get_config()
        config.update({
            'num_patches': self.num_patches,
            'projection': self.projection,
            'position_embedding': self.position_embedding,

        })
        return config

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [ ]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0

):
    inputs = Input(shape=input_shape)

    x = inputs
    embedding_layer = PatchEncoder(num_patches,projection_dim)
    x = embedding_layer(x)
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
        outputs = layers.Dense(7, activation="softmax")(x)
    return  keras.Model(inputs, outputs)

In [ ]:
input_shape = input_shape

model = build_model(
    input_shape,
    head_size=256,
    num_heads=8,
    ff_dim=64,
    num_transformer_blocks=2,
    mlp_units=[2048],
    mlp_dropout=0.4,
    dropout=0.25) #0.25

#model.compile(
   # loss="sparse_categorical_crossentropy",
    #optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    #metrics=["sparse_categorical_accuracy"],
#)
model.summary()

# Conv1D

In [ ]:
def build_model(X_train, n_outputs):
    # Input layer
    input_layer = Input(shape=X_train.shape[1:])

    # Convolutional layers
    conv_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(input_layer)
    conv_layer2 = Conv1D(filters=128, kernel_size=3, activation='relu')(conv_layer)
    conv_layer3 = Conv1D(filters=128, kernel_size=5, activation='relu')(conv_layer)

    # Pooling layer
    pooling = MaxPooling1D(pool_size=2)(conv_layer)

    # Further convolutional layers
    conv_layer4 = Conv1D(filters=128, kernel_size=1, activation='relu')(pooling)

    # Concatenation of layer outputs
    concat = Concatenate(axis=1)
    conca_layers = concat([conv_layer2, conv_layer3, conv_layer4])

    # Dropout layers
    Dplayer1 = Dropout(0.4)(conca_layers)
    conv_layer5 = Conv1D(filters=128, kernel_size=3, activation='relu')(Dplayer1)
    Dplayer2 = Dropout(0.4)(conv_layer5)
    conv_layer6 = Conv1D(filters=256, kernel_size=3, activation='relu')(Dplayer2)
    Dplayer3 = Dropout(0.4)(conv_layer6)

    # Flatten for dense layer
    flatten_layer = Flatten()(Dplayer3)
    dense_layer1 = Dense(units=512, activation='relu')(flatten_layer)
    dense_layer2 = Dropout(0.5)(dense_layer1)

    # Output layer
    output_layer = Dense(units=n_outputs, activation='softmax')(dense_layer2)

    # Create model
    model = Model(inputs=input_layer, outputs=output_layer)

    return model

# Conv1D-RF

In [ ]:
def build_and_train_model(X_train, y_train, n_outputs):
    # Define the model architecture
    input_layer = Input(shape=X_train.shape[1:])
    conv_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(input_layer)
    conv_layer2 = Conv1D(filters=128, kernel_size=3, activation='relu')(conv_layer)
    conv_layer3 = Conv1D(filters=128, kernel_size=5, activation='relu')(conv_layer)
    pooling = MaxPooling1D(pool_size=2)(conv_layer)
    conv_layer4 = Conv1D(filters=128, kernel_size=1, activation='relu')(pooling)

    concat = Concatenate(axis=1)([conv_layer2, conv_layer3, conv_layer4])
    dropout1 = Dropout(0.4)(concat)
    conv_layer5 = Conv1D(filters=128, kernel_size=3, activation='relu')(dropout1)
    dropout2 = Dropout(0.4)(conv_layer5)
    conv_layer6 = Conv1D(filters=256, kernel_size=3, activation='relu')(dropout2)
    dropout3 = Dropout(0.4)(conv_layer6)

    flatten_layer = Flatten()(dropout3)
    dense_layer1 = Dense(units=512, activation='relu')(flatten_layer)
    dense_layer2 = Dropout(0.5)(dense_layer1)
    output_layer = Dense(units=n_outputs, activation='softmax')(dense_layer2)
    model = Model(inputs=input_layer, outputs=output_layer)

    # Assuming a compiled model is needed, compile here (this step may vary)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Load the pre-trained model
    new_model = load_model('gdrive/My Drive/My Model/conv1D-mchi(all).h5')

    # Extract features from the model
    layer_name = 'dropout_49'
    FC_layer_model = Model(inputs=new_model.input, outputs=new_model.get_layer(layer_name).output)
    output_feature = FC_layer_model.predict(X_train)

    # Create DataFrame for extracted features
    X_RF_col = ["f_" + str(i) for i in range(512)]
    train_feature2 = pd.DataFrame(data=output_feature, columns=X_RF_col)

    # Train a RandomForest Classifier
    Conv1DRF_model = RandomForestClassifier(n_estimators=500, random_state=41, n_jobs=-1, min_samples_split=4)
    Conv1DRF_model.fit(train_feature2, y_train)

    return Conv1DRF_model

# Usage:
# model = build_and_train_model(X_train, y_train, n_outputs)

# Conv1D-LSTM

In [ ]:
def attention_3d_block(hidden_states):
    hidden_size = int(hidden_states.shape[2])
    score_first_part = Dense(hidden_size, use_bias=False, name='attention_score_vec')(hidden_states)
    h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,), name='last_hidden_state')(hidden_states)
    score = dot([score_first_part, h_t], [2, 1], name='attention_score')
    attention_weights = Activation('softmax', name='attention_weight')(score)
    context_vector = dot([hidden_states, attention_weights], [1, 1], name='context_vector')
    pre_activation = concatenate([context_vector, h_t], name='attention_output')
    attention_vector = Dense(256, use_bias=False, activation='softmax', name='attention_vector')(pre_activation)
    return attention_vector

def m2_block(init, filter, kernel):
    x = init
    conv1 = Conv1D(filter, kernel, padding='same', kernel_initializer='he_normal', activation='relu')(x)
    skip = conv1
    conv2 = Conv1D(filter, kernel, padding='same', kernel_initializer='he_normal', activation='relu')(conv1)
    conv3 = Conv1D(filter, kernel, padding='same', kernel_initializer='he_normal', activation='relu')(conv2)
    x = BatchNormalization()(conv3)
    x = Add()([x, skip])
    x = Activation('relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Flatten()(x)
    out = Dense(512, activation="relu")(x)
    out = Dropout(0.5)(out)
    return out

def model1(init):
    x = Bidirectional(CuDNNLSTM(32, return_sequences=True))(init)
    x = attention_3d_block(x)
    out = Dense(512, activation="relu")(x)
    return out

def model2(init):
    x0 = m2_block(init, 64, 3)
    x1 = m2_block(init, 128, 3)
    x2 = m2_block(init, 256, 5)
    x = concatenate([x0, x1, x2])
    x = Dropout(0.5)(x)
    out = Dense(512, activation="relu")(x)
    return out

def get_complete_model(X_train, n_outputs):
    inp = Input(X_train.shape[1:])
    out1 = model1(inp)
    out2 = model2(inp)
    conc = concatenate([out1, out2])
    x = Dense(512, activation='relu')(conc)
    x = Dropout(0.5)(x)
    outp = Dense(n_outputs, activation="softmax")(x)
    model = Model(inputs=inp, outputs=outp)
    return model

In [ ]:
# visualising constructed models

In [ ]:
import keras
import pydot as pyd
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

keras.utils.vis_utils.pydot = pyd

#Visualize Model

def visualize_model(model):
    return SVG(model_to_dot(model,show_shapes= True, show_layer_names=True, dpi=65).create(prog='dot', format='svg'))
#create your model
#then call the function on your model

visualize_model(model)

In [ ]:
# Training with early stopping

In [ ]:
early_stop = EarlyStopping(monitor = 'val_loss',
                            mode = 'min',
                            min_delta = 0,
                            patience = 10,
                            restore_best_weights = True)

checkpoint = ModelCheckpoint(filepath = 'F:/weight/conv1DLSTM.h5',
                             monitor = 'val_loss',
                             mode ='min',
                             save_best_only = True)

tensorboard = TensorBoard(log_dir='SA_logs/{}'.format(time()))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, decay=5e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
hist = model.fit(X_train,
                 y_train,
                 epochs = 100,
                 batch_size = 128,
                 validation_data = (X_val, y_val),
                 #shuffle=False,
                 #class_weight=class_weights,
                 callbacks=[early_stop,
                            checkpoint,
                            tensorboard])